## 📦 1. Installation et Imports

In [ ]:
# Installation des packages
!pip install -q datasets transformers scikit-learn kagglehub

# Télécharger NLTK data
import nltk
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

print("✅ Installation terminée!")

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pickle
import re
from collections import Counter

from datasets import load_dataset
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Configuration
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Imports réussis!")

## 📂 2. Créer la Structure de Dossiers

In [ ]:
# Créer les dossiers nécessaires
folders = [
    'data/raw',
    'data/processed',
    'models/lstm',
    'models/bilstm',
    'models/cnn_bilstm',
    'models/bert',
    'results/figures',
    'results/metrics'
]

for folder in folders:
    Path(folder).mkdir(parents=True, exist_ok=True)

print("✅ Structure de dossiers créée!")
!ls -la

## 🔗 3. Téléchargement du Dataset GoEmotions depuis Kaggle

**⚠️ IMPORTANT : Personne 1 doit faire cette étape**

**Source** : https://www.kaggle.com/datasets/debarshichanda/goemotions/data

### Méthode 1 : Chargement Automatique avec KaggleHub (Recommandé ✅)
Utilise l'API Kaggle pour télécharger automatiquement

In [ ]:
# ==========================================
# MÉTHODE 1 : Chargement Automatique (Recommandé)
# ==========================================
import kagglehub
import pandas as pd
import os

print("📥 Téléchargement automatique du dataset GoEmotions depuis Kaggle...")
print("⏳ Cela peut prendre 1-2 minutes...\n")

try:
    # Télécharger le dataset (nouvelle API)
    path = kagglehub.dataset_download("debarshichanda/goemotions")
    
    print(f"✅ Dataset téléchargé dans: {path}")
    
    # Lister les fichiers disponibles
    files_in_dataset = os.listdir(path)
    print(f"\n📁 Fichiers disponibles: {files_in_dataset}")
    
    # Trouver le fichier CSV principal
    csv_files = [f for f in files_in_dataset if f.endswith('.csv')]
    
    if csv_files:
        # Charger le premier fichier CSV trouvé
        csv_file = csv_files[0]
        file_path = os.path.join(path, csv_file)
        
        print(f"\n📊 Chargement du fichier: {csv_file}")
        df = pd.read_csv(file_path)
        
        print(f"\n✅ Dataset chargé avec succès!")
        print(f"📊 Statistiques:")
        print(f"  Nombre total d'échantillons: {len(df):,}")
        print(f"  Nombre de colonnes: {len(df.columns)}")
        
        # Sauvegarder dans data/raw/
        df.to_csv('data/raw/goemotions.csv', index=False)
        print("\n💾 Dataset sauvegardé dans data/raw/goemotions.csv")
        
        print(f"\n📋 Colonnes disponibles:")
        print(df.columns.tolist())
        
        print(f"\n👁️ Aperçu des 5 premières lignes:")
        display(df.head())
    else:
        print("❌ Aucun fichier CSV trouvé dans le dataset")
        print("💡 Utilisez la Méthode 2 (Upload Manuel) ci-dessous")
        
except Exception as e:
    print(f"\n❌ Erreur lors du chargement automatique: {e}")
    print("\n💡 Solution : Utilisez la Méthode 2 (Upload Manuel) ci-dessous")

In [ ]:
# ==========================================
# MÉTHODE 2 : Upload Manuel (Alternative)
# ==========================================
# ⚠️ Exécutez cette cellule si la Méthode 1 a échoué

from google.colab import files
import pandas as pd

print("📤 Veuillez uploader les fichiers CSV du dataset GoEmotions depuis Kaggle...")
print("👉 Téléchargez d'abord depuis: https://www.kaggle.com/datasets/debarshichanda/goemotions/data")
print("👉 Le dataset peut être en 1 ou 3 fichiers (goemotions.csv OU goemotions_1/2/3.csv)")
print("👉 Uploadez TOUS les fichiers CSV, puis cliquez sur 'Choisir les fichiers' ci-dessous\n")

uploaded = files.upload()

# Trouver tous les fichiers CSV uploadés
csv_files = [name for name in uploaded.keys() if name.endswith('.csv')]

if csv_files:
    print(f"\n✅ {len(csv_files)} fichier(s) CSV détecté(s): {csv_files}")
    
    # Charger tous les fichiers CSV et les combiner
    print("📊 Chargement des données en cours...")
    dfs = []
    for csv_file in sorted(csv_files):  # Trier pour avoir 1, 2, 3 dans l'ordre
        print(f"  - Chargement de {csv_file}...")
        df_temp = pd.read_csv(csv_file)
        dfs.append(df_temp)
        print(f"    ✓ {len(df_temp):,} lignes chargées")
    
    # Combiner tous les DataFrames
    if len(dfs) > 1:
        print(f"\n🔗 Combinaison de {len(dfs)} fichiers...")
        df = pd.concat(dfs, ignore_index=True)
        print(f"✅ Fichiers combinés!")
    else:
        df = dfs[0]
    
    # Sauvegarder dans data/raw/
    df.to_csv('data/raw/goemotions.csv', index=False)
    
    print(f"\n✅ Dataset chargé avec succès!")
    print(f"📊 Statistiques:")
    print(f"  Nombre total d'échantillons: {len(df):,}")
    print(f"  Nombre de colonnes: {len(df.columns)}")
    print(f"\n📋 Colonnes disponibles:")
    print(df.columns.tolist())
    print(f"\n👁️ Aperçu des premières lignes:")
    display(df.head())
else:
    print("❌ ERREUR : Aucun fichier CSV uploadé!")
    print("👉 Téléchargez le dataset depuis: https://www.kaggle.com/datasets/debarshichanda/goemotions/data")
    print("👉 Puis réexécutez cette cellule")


In [ ]:
# ==========================================
# MÉTHODE 3 : Charger depuis les fichiers déjà uploadés
# ==========================================
# ⚠️ Si vous avez déjà uploadé les fichiers et ils sont dans /content/

import pandas as pd
import os

# Liste des fichiers CSV à charger (modifiez les chemins si nécessaire)
csv_files = [
    '/content/goemotions_1.csv',
    '/content/goemotions_2.csv',
    '/content/goemotions_3.csv'
]

# Vérifier quels fichiers existent
existing_files = [f for f in csv_files if os.path.exists(f)]

if existing_files:
    print(f"✅ {len(existing_files)} fichier(s) trouvé(s)!")
    
    # Charger tous les fichiers
    dfs = []
    for csv_file in existing_files:
        print(f"📊 Chargement de {csv_file}...")
        df_temp = pd.read_csv(csv_file)
        dfs.append(df_temp)
        print(f"  ✓ {len(df_temp):,} lignes chargées")
    
    # Combiner tous les DataFrames
    if len(dfs) > 1:
        print(f"\n🔗 Combinaison de {len(dfs)} fichiers...")
        df = pd.concat(dfs, ignore_index=True)
        print(f"✅ Fichiers combinés!")
    else:
        df = dfs[0]
    
    # Sauvegarder dans data/raw/
    df.to_csv('data/raw/goemotions.csv', index=False)
    
    print(f"\n✅ Dataset chargé avec succès!")
    print(f"📊 Statistiques:")
    print(f"  Nombre total d'échantillons: {len(df):,}")
    print(f"  Nombre de colonnes: {len(df.columns)}")
    print(f"\n📋 Colonnes disponibles:")
    print(df.columns.tolist())
    print(f"\n👁️ Aperçu des premières lignes:")
    display(df.head())
else:
    print("❌ Aucun fichier trouvé!")
    print("💡 Vérifiez les chemins des fichiers. Exécutez cette commande pour voir où sont vos fichiers:")
    print("!ls -la /content/*.csv")

### Méthode 2 : Upload Manuel (Si la Méthode 1 échoue)

**Instructions :**
1. Aller sur https://www.kaggle.com/datasets/debarshichanda/goemotions/data
2. Cliquer sur **"Download"** pour télécharger goemotions.csv sur votre PC
3. Exécuter la cellule ci-dessous et cliquer sur **"Choisir les fichiers"**
4. Sélectionner le fichier goemotions.csv téléchargé

In [ ]:
# ==========================================
# ÉTAPE 2 : Vérification du format du dataset
# ==========================================

print("🔍 Analyse du format du dataset...\n")

# Vérifier les colonnes
print(f"📊 Informations du dataset:")
print(f"  Shape: {df.shape}")
print(f"  Colonnes: {list(df.columns)}")

# Afficher quelques exemples
print(f"\n📝 Exemples de données:")
display(df.head(10))

# Vérifier les statistiques
print(f"\n📈 Statistiques de base:")
print(df.describe())

In [ ]:
# ==========================================
# ÉTAPE 3 : Préparation du dataset pour l'entraînement
# ==========================================

print("🔧 Préparation du dataset au format multi-label...\n")

# Identifier la colonne de texte
text_col = None
for col in ['text', 'comment_text', 'sentence', 'content']:
    if col in df.columns:
        text_col = col
        break

if text_col is None:
    print("❌ Colonne de texte non trouvée!")
    print(f"Colonnes disponibles: {df.columns.tolist()}")
else:
    print(f"✅ Colonne de texte identifiée: '{text_col}'")

# Identifier les colonnes d'émotions (28 émotions GoEmotions)
emotion_labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

emotion_cols = [col for col in df.columns if col in emotion_labels]

print(f"\n🎭 Colonnes d'émotions trouvées: {len(emotion_cols)}/{len(emotion_labels)}")
if len(emotion_cols) > 0:
    print(f"Émotions: {emotion_cols[:5]}... (affichant 5 premières)")

# Créer les splits train/val/test (80/10/10)
if len(emotion_cols) >= 27 and text_col is not None:
    print("\n✅ Dataset au format multi-label! Création des splits...")
    
    from sklearn.model_selection import train_test_split
    
    # Split: 80% train, 10% val, 10% test
    train_data, temp_data = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
    val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, shuffle=True)
    
    print(f"\n📊 Splits créés:")
    print(f"  Train:      {len(train_data):,} samples ({len(train_data)/len(df)*100:.1f}%)")
    print(f"  Validation: {len(val_data):,} samples ({len(val_data)/len(df)*100:.1f}%)")
    print(f"  Test:       {len(test_data):,} samples ({len(test_data)/len(df)*100:.1f}%)")
    print(f"  Total:      {len(df):,} samples")
    
    # Créer les DataFrames finaux
    train_df = train_data.copy()
    val_df = val_data.copy()
    test_df = test_data.copy()
    
    # Normaliser la colonne 'text'
    if text_col != 'text':
        train_df['text'] = train_df[text_col]
        val_df['text'] = val_df[text_col]
        test_df['text'] = test_df[text_col]
    
    # Créer la colonne 'labels' (liste des indices des émotions actives)
    def get_label_indices(row):
        return [i for i, col in enumerate(emotion_labels) if col in emotion_cols and row.get(col, 0) == 1]
    
    print("\n🏷️ Création de la colonne 'labels'...")
    train_df['labels'] = train_df.apply(get_label_indices, axis=1)
    val_df['labels'] = val_df.apply(get_label_indices, axis=1)
    test_df['labels'] = test_df.apply(get_label_indices, axis=1)
    
    print("✅ Dataset prêt pour l'entraînement!")
    print(f"\n📋 Exemple de labels:")
    print(f"  Texte: {train_df.iloc[0]['text'][:80]}...")
    print(f"  Labels: {train_df.iloc[0]['labels']}")
    print(f"  Émotions: {[emotion_labels[i] for i in train_df.iloc[0]['labels']]}")
    
else:
    print("\n⚠️ Format du dataset non reconnu. Utilisation de HuggingFace comme fallback...")
    from datasets import load_dataset
    dataset = load_dataset('go_emotions', 'simplified')
    train_df = pd.DataFrame(dataset['train'])
    val_df = pd.DataFrame(dataset['validation'])
    test_df = pd.DataFrame(dataset['test'])
    emotion_labels = [
        'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
        'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
        'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
        'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
        'relief', 'remorse', 'sadness', 'surprise', 'neutral'
    ]
    print(f"✅ Dataset HuggingFace chargé!")
    print(f"  Train:      {len(train_df):,} samples")
    print(f"  Validation: {len(val_df):,} samples")
    print(f"  Test:       {len(test_df):,} samples")

NUM_CLASSES = len(emotion_labels)
print(f"\n🎯 Nombre de classes: {NUM_CLASSES}")

### 📊 Préparation du Dataset pour le Projet

Le dataset Kaggle GoEmotions contient toutes les émotions dans une seule colonne.
Nous devons le transformer en format multi-label (28 colonnes binaires).

In [ ]:
# Afficher un aperçu
print("\n📝 Aperçu des données:")
train_df.head()

## 🏷️ 4. Labels des Émotions

In [ ]:
# Labels d'émotions GoEmotions
emotion_labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

NUM_CLASSES = len(emotion_labels)

print(f"\n🎭 Nombre d'émotions: {NUM_CLASSES}")
print(f"\nListe des émotions:")
for i, emotion in enumerate(emotion_labels, 1):
    print(f"  {i:2d}. {emotion}")

## 📊 5. Analyse Exploratoire

In [ ]:
# Distribution des émotions
all_labels = []
for labels in train_df['labels']:
    all_labels.extend(labels)

label_counts = Counter(all_labels)
label_counts_sorted = sorted(label_counts.items())

# Préparer les données pour le graphique
indices = [x[0] for x in label_counts_sorted]
counts = [x[1] for x in label_counts_sorted]
labels_names = [emotion_labels[i] if i < len(emotion_labels) else f"Label {i}" for i in indices]

# Plot
fig, ax = plt.subplots(figsize=(16, 6))
bars = ax.bar(labels_names, counts, color='steelblue', edgecolor='navy')

# Colorer les barres selon la fréquence
colors = plt.cm.viridis(np.linspace(0, 1, len(counts)))
for bar, color in zip(bars, colors):
    bar.set_color(color)

ax.set_xlabel('Émotion', fontsize=12, fontweight='bold')
ax.set_ylabel('Fréquence', fontsize=12, fontweight='bold')
ax.set_title('Distribution des Émotions dans le Training Set', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('results/figures/emotion_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📈 Statistiques:")
print(f"  Total de labels: {sum(counts):,}")
print(f"  Émotion la plus fréquente: {labels_names[counts.index(max(counts))]} ({max(counts):,} occurrences)")
print(f"  Émotion la moins fréquente: {labels_names[counts.index(min(counts))]} ({min(counts):,} occurrences)")
print(f"  Ratio déséquilibre: {max(counts)/min(counts):.2f}")

In [ ]:
# Nombre de labels par échantillon
labels_per_sample = [len(labels) for labels in train_df['labels']]

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogramme
axes[0].hist(labels_per_sample, bins=range(1, max(labels_per_sample)+2), 
             edgecolor='black', color='coral', alpha=0.7)
axes[0].set_xlabel('Nombre de Labels par Échantillon', fontweight='bold')
axes[0].set_ylabel('Fréquence', fontweight='bold')
axes[0].set_title('Distribution du Nombre de Labels', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Boxplot
axes[1].boxplot(labels_per_sample, vert=True)
axes[1].set_ylabel('Nombre de Labels', fontweight='bold')
axes[1].set_title('Boxplot du Nombre de Labels', fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('results/figures/labels_per_sample.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📊 Statistiques Multi-Label:")
print(f"  Moyenne: {np.mean(labels_per_sample):.2f} labels/échantillon")
print(f"  Médiane: {np.median(labels_per_sample):.0f} labels/échantillon")
print(f"  Maximum: {max(labels_per_sample)} labels/échantillon")
print(f"  Minimum: {min(labels_per_sample)} labels/échantillon")

In [ ]:
# Longueur des textes
train_df['text_length'] = train_df['text'].apply(len)
train_df['word_count'] = train_df['text'].apply(lambda x: len(str(x).split()))

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].hist(train_df['text_length'], bins=50, edgecolor='black', color='lightblue')
axes[0].axvline(train_df['text_length'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Moyenne: {train_df["text_length"].mean():.0f}')
axes[0].set_xlabel('Longueur du Texte (caractères)', fontweight='bold')
axes[0].set_ylabel('Fréquence', fontweight='bold')
axes[0].set_title('Distribution de la Longueur des Textes', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].hist(train_df['word_count'], bins=50, edgecolor='black', color='lightgreen')
axes[1].axvline(train_df['word_count'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Moyenne: {train_df["word_count"].mean():.1f}')
axes[1].set_xlabel('Nombre de Mots', fontweight='bold')
axes[1].set_ylabel('Fréquence', fontweight='bold')
axes[1].set_title('Distribution du Nombre de Mots', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/figures/text_length_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📏 Statistiques de Longueur:")
print(f"  Longueur moyenne: {train_df['text_length'].mean():.1f} caractères")
print(f"  Nombre de mots moyen: {train_df['word_count'].mean():.1f} mots")
print(f"  Percentile 95: {train_df['word_count'].quantile(0.95):.0f} mots")

In [ ]:
print("\n📊 ANALYSE DE CO-OCCURRENCE DES ÉMOTIONS")
print("="*60)

# Créer la matrice de co-occurrence
# D'abord préparer y_train si pas encore fait (pour cette section)
if 'y_train' not in locals():
    y_train_temp = prepare_labels(train_df['labels'].tolist(), NUM_CLASSES)
else:
    y_train_temp = y_train

co_occurrence = np.dot(y_train_temp.T, y_train_temp)

# Normaliser pour avoir des probabilités
co_occurrence_prob = co_occurrence / np.diag(co_occurrence)[:, None]

# Créer un DataFrame pour meilleure visualisation
co_occurrence_df = pd.DataFrame(
    co_occurrence_prob,
    index=emotion_labels,
    columns=emotion_labels
)

# Visualiser la matrice
plt.figure(figsize=(16, 14))
sns.heatmap(
    co_occurrence_df,
    cmap='YlOrRd',
    annot=False,  # Trop de données pour annoter
    fmt='.2f',
    cbar_kws={'label': 'Probabilité de co-occurrence'},
    linewidths=0.5,
    square=True
)
plt.title('Matrice de Co-occurrence des Émotions\n(Probabilité qu\'une émotion apparaisse avec une autre)', 
          fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Émotion', fontsize=12, fontweight='bold')
plt.ylabel('Émotion', fontsize=12, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('results/figures/emotion_cooccurrence.png', dpi=300, bbox_inches='tight')
plt.show()

# Trouver les paires d'émotions les plus corrélées
print("\n🔗 TOP 10 PAIRES D'ÉMOTIONS LES PLUS CORRÉLÉES:")
print("-"*60)

correlations = []
for i in range(NUM_CLASSES):
    for j in range(i+1, NUM_CLASSES):
        correlations.append({
            'Emotion1': emotion_labels[i],
            'Emotion2': emotion_labels[j],
            'Correlation': co_occurrence_prob[i, j]
        })

correlations_df = pd.DataFrame(correlations).sort_values('Correlation', ascending=False)
print(correlations_df.head(10).to_string(index=False))

print("\n✅ Analyse de co-occurrence terminée!")

### 📊 5.5 Matrice de Co-occurrence des Émotions

In [ ]:
# Calculer les ratios de déséquilibre
print("\n📊 ANALYSE DU DÉSÉQUILIBRE DES CLASSES")
print("="*60)

max_count = max(counts)
min_count = min(counts)
imbalance_ratio = max_count / min_count

print(f"\nRatio de déséquilibre : {imbalance_ratio:.2f}:1")
print(f"Classe majoritaire : {labels_names[counts.index(max_count)]} ({max_count:,} occurrences)")
print(f"Classe minoritaire : {labels_names[counts.index(min_count)]} ({min_count:,} occurrences)")
print("="*60)

# Calculer les poids de classe pour gérer le déséquilibre
emotion_weights = {}
for emotion, count in zip(emotion_labels, counts):
    weight = len(train_df) / (NUM_CLASSES * count)
    emotion_weights[emotion] = weight

# Créer un DataFrame des poids
weights_df = pd.DataFrame({
    'Emotion': emotion_labels,
    'Count': counts,
    'Weight': [emotion_weights[em] for em in emotion_labels]
}).sort_values('Weight', ascending=False)

print(f"\n⚖️ POIDS DES CLASSES (Top 10 et Bottom 10):")
print(f"\nTop 10 (classes sous-représentées - poids élevés):")
print(weights_df.head(10).to_string(index=False))
print(f"\nBottom 10 (classes sur-représentées - poids faibles):")
print(weights_df.tail(10).to_string(index=False))

# Visualiser les poids
fig, ax = plt.subplots(figsize=(14, 6))
bars = ax.bar(weights_df['Emotion'], weights_df['Weight'], alpha=0.7, edgecolor='black')

# Colorer selon le poids
colors = plt.cm.RdYlGn_r(np.linspace(0, 1, len(weights_df)))
for bar, color in zip(bars, colors):
    bar.set_color(color)

ax.set_xlabel('Émotion', fontsize=12, fontweight='bold')
ax.set_ylabel('Poids de Classe', fontsize=12, fontweight='bold')
ax.set_title('Poids des Classes pour Gérer le Déséquilibre', fontsize=14, fontweight='bold')
ax.axhline(y=1.0, color='red', linestyle='--', linewidth=2, label='Poids équilibré (1.0)')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('results/figures/class_weights.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Analyse du déséquilibre terminée!")

### 📊 5.4 Analyse Approfondie du Déséquilibre des Classes

## 🧹 6. Prétraitement des Textes

In [ ]:
def clean_text(text):
    """Nettoyer le texte"""
    if not isinstance(text, str):
        return ""
    
    # Lowercase
    text = text.lower()
    
    # Supprimer URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    
    # Supprimer mentions et hashtags
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    
    # Garder seulement lettres, chiffres et ponctuation basique
    text = re.sub(r'[^a-zA-Z0-9\s.,!?\'\-]', '', text)
    
    # Supprimer espaces multiples
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Appliquer le nettoyage
print("Nettoyage des textes...")
train_df['text_clean'] = train_df['text'].apply(clean_text)
val_df['text_clean'] = val_df['text'].apply(clean_text)
test_df['text_clean'] = test_df['text'].apply(clean_text)

print("\n✅ Nettoyage terminé!")
print("\nExemple de transformation:")
print(f"Avant: {train_df['text'].iloc[0]}")
print(f"Après: {train_df['text_clean'].iloc[0]}")

## 🔢 7. Tokenization et Séquences

In [ ]:
# Paramètres
MAX_VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 128

print(f"Configuration:")
print(f"  Taille du vocabulaire: {MAX_VOCAB_SIZE:,}")
print(f"  Longueur max des séquences: {MAX_SEQUENCE_LENGTH}")

# Créer le tokenizer
print("\nCréation du tokenizer...")
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token='<OOV>')
tokenizer.fit_on_texts(train_df['text_clean'])

print(f"\n✅ Tokenizer créé!")
print(f"  Vocabulaire complet: {len(tokenizer.word_index):,} mots")
print(f"  Vocabulaire utilisé: {MAX_VOCAB_SIZE:,} mots")

In [ ]:
# Convertir en séquences
print("Conversion en séquences...")
X_train_seq = tokenizer.texts_to_sequences(train_df['text_clean'])
X_val_seq = tokenizer.texts_to_sequences(val_df['text_clean'])
X_test_seq = tokenizer.texts_to_sequences(test_df['text_clean'])

# Padding
print("Application du padding...")
X_train = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH, 
                        padding='post', truncating='post')
X_val = pad_sequences(X_val_seq, maxlen=MAX_SEQUENCE_LENGTH, 
                      padding='post', truncating='post')
X_test = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH, 
                       padding='post', truncating='post')

print(f"\n✅ Séquences créées!")
print(f"  X_train: {X_train.shape}")
print(f"  X_val:   {X_val.shape}")
print(f"  X_test:  {X_test.shape}")

## 🏷️ 8. Préparer les Labels Multi-Label

In [ ]:
def prepare_labels(labels_list, num_classes=28):
    """
    Convertir les listes de labels en matrices binaires
    """
    n_samples = len(labels_list)
    label_matrix = np.zeros((n_samples, num_classes), dtype=np.float32)
    
    for i, labels in enumerate(labels_list):
        for label_idx in labels:
            if label_idx < num_classes:
                label_matrix[i, label_idx] = 1.0
    
    return label_matrix

# Préparer les labels
print("Préparation des labels...")
y_train = prepare_labels(train_df['labels'].tolist(), NUM_CLASSES)
y_val = prepare_labels(val_df['labels'].tolist(), NUM_CLASSES)
y_test = prepare_labels(test_df['labels'].tolist(), NUM_CLASSES)

print(f"\n✅ Labels préparés!")
print(f"  y_train: {y_train.shape}")
print(f"  y_val:   {y_val.shape}")
print(f"  y_test:  {y_test.shape}")


# ==========================================
# ÉTAPE 8.1 : Gestion du Déséquilibre des Classes (Requis pour l'énoncé)
# ==========================================
print("\n⚖️ Calcul des poids de classes pour gérer le déséquilibre...")

# Calculer le nombre d'occurrences positives pour chaque classe
class_counts = y_train.sum(axis=0)
total_samples = len(y_train)

# Calculer les poids (plus une classe est rare, plus son poids est élevé)
# Formule standard pour multi-label: pos_weight = (total - pos) / pos
class_weights = {}
pos_weights = []

print("Poids calculés (Top 5 rares vs fréquents):")
sorted_indices = np.argsort(class_counts)

for i in range(NUM_CLASSES):
    pos = max(class_counts[i], 1) # Éviter division par 0
    neg = total_samples - pos
    weight = neg / pos
    class_weights[i] = weight
    pos_weights.append(weight)

# Afficher quelques exemples
print("\nClasses les plus rares (Poids élevés):")
for i in sorted_indices[:5]:
    print(f"  {emotion_labels[i]}: Count={int(class_counts[i])}, Weight={class_weights[i]:.2f}")

print("\nClasses les plus fréquentes (Poids faibles):")
for i in sorted_indices[-5:]:
    print(f"  {emotion_labels[i]}: Count={int(class_counts[i])}, Weight={class_weights[i]:.2f}")


# ==========================================
# ÉTAPE 8.2 : Préparation des Embeddings GloVe (Requis pour l'étude d'ablation)
# ==========================================
# Note: L'énoncé demande de comparer les embeddings. Nous préparons GloVe ici.
print("\n🌍 Préparation des Embeddings GloVe (Pre-trained)...")

# Fonction pour charger GloVe
def load_glove_embeddings(vocab_word_index, embedding_dim=100):
    print("  📥 Téléchargement de GloVe (glove.6B.100d)... Cela peut prendre quelques instants.")
    
    # Vérifier si le fichier existe déjà
    if not os.path.exists('glove.6B.100d.txt'):
        print("  Downloading GloVe...")
        !wget -q --no-check-certificate http://nlp.stanford.edu/data/glove.6B.zip
        !unzip -q glove.6B.zip
    
    print("  Chargement des vecteurs en mémoire...")
    embeddings_index = {}
    with open('glove.6B.100d.txt', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    print(f"  ✅ {len(embeddings_index):,} vecteurs de mots trouvés dans GloVe.")

    # Créer la matrice d'embedding seulement pour notre vocabulaire
    num_words = min(MAX_VOCAB_SIZE, len(vocab_word_index) + 1)
    embedding_matrix = np.zeros((num_words, embedding_dim))
    
    hits = 0
    misses = 0
    
    for word, i in vocab_word_index.items():
        if i >= MAX_VOCAB_SIZE:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # Les mots non trouvés dans l'embedding index seront tous des zéros.
            embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            misses += 1
            
    print(f"  ✅ Matrice construite: {hits} mots trouvés, {misses} mots manquants.")
    return embedding_matrix

# Préparer la matrice au cas où on voudrait l'utiliser
try:
    embedding_matrix = load_glove_embeddings(tokenizer.word_index, embedding_dim=128) # Note: GloVe est 100d, on adaptera ou on utilisera 100d
    # Note: Si on utilise glove.6B.100d, la dimension DOIT être 100.
    # Pour simplifier et matcher GloVe standard, on refait avec dim=100
    embedding_matrix = load_glove_embeddings(tokenizer.word_index, embedding_dim=100)
    USE_GLOVE = True
except Exception as e:
    print(f"⚠️ Impossible de charger GloVe (Erreur: {e}). On continuera sans.")
    embedding_matrix = None
    USE_GLOVE = False

## 💾 9. Sauvegarder les Données Préparées

In [ ]:
print("Sauvegarde des données préparées...\n")

# Sauvegarder les séquences
np.save('data/processed/X_train.npy', X_train)
np.save('data/processed/X_val.npy', X_val)
np.save('data/processed/X_test.npy', X_test)
print("✅ Séquences sauvegardées")

# Sauvegarder les labels
np.save('data/processed/y_train.npy', y_train)
np.save('data/processed/y_val.npy', y_val)
np.save('data/processed/y_test.npy', y_test)
print("✅ Labels sauvegardés")

# Sauvegarder le tokenizer
with open('data/processed/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
print("✅ Tokenizer sauvegardé")

# Sauvegarder les métadonnées
metadata = {
    'emotion_labels': emotion_labels,
    'num_classes': NUM_CLASSES,
    'max_vocab_size': MAX_VOCAB_SIZE,
    'max_sequence_length': MAX_SEQUENCE_LENGTH,
    'vocab_size': len(tokenizer.word_index),
    'train_size': len(X_train),
    'val_size': len(X_val),
    'test_size': len(X_test),
    'use_glove': USE_GLOVE
}

with open('data/processed/metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)
print("✅ Métadonnées sauvegardées")

# Sauvegarder les poids de classes (NOUVEAU - Requis Partie 1)
with open('data/processed/class_weights.pkl', 'wb') as f:
    pickle.dump(class_weights, f)
print("✅ Poids des classes sauvegardés (pour gérer le déséquilibre)")

# Sauvegarder la matrice d'embedding GloVe (NOUVEAU - Requis Partie 4)
if USE_GLOVE and embedding_matrix is not None:
    np.save('data/processed/embedding_matrix.npy', embedding_matrix)
    print("✅ Matrice d'embedding GloVe sauvegardée")
else:
    print("⚠️ Matrice GloVe non sauvegardée (non chargée)")

# Sauvegarder les textes originaux pour BERT (qui a besoin des textes bruts)
with open('data/processed/texts_train.pkl', 'wb') as f:
    pickle.dump(train_df['text_clean'].tolist(), f)
print("✅ Textes train sauvegardés")

with open('data/processed/texts_val.pkl', 'wb') as f:
    pickle.dump(val_df['text_clean'].tolist(), f)
print("✅ Textes val sauvegardés")

with open('data/processed/texts_test.pkl', 'wb') as f:
    pickle.dump(test_df['text_clean'].tolist(), f)
print("✅ Textes test sauvegardés")

print("\n" + "="*60)
print("🎉 PRÉPARATION DES DONNÉES TERMINÉE !")
print("="*60)
print("\nVous pouvez maintenant exécuter les notebooks d'entraînement:")
print("  📓 Notebook_1_LSTM.ipynb")
print("  📓 Notebook_2_BiLSTM_Attention.ipynb")
print("  📓 Notebook_3_CNN_BiLSTM.ipynb")
print("  📓 Notebook_4_BERT.ipynb")
print("\nPuis pour la comparaison:")
print("  📓 Notebook_5_Comparaison_Finale.ipynb")

In [ ]:
# Vérifier les fichiers créés
print("\n📁 Fichiers créés dans data/processed/:")
!ls -lh data/processed/

## 📤 10. [Optionnel] Sauvegarder vers Google Drive

In [ ]:
# Monter Google Drive (optionnel)
from google.colab import drive
drive.mount('/content/drive')

# Créer un dossier dans Drive
!mkdir -p "/content/drive/MyDrive/emotion_detection_project"

# Copier les données
!cp -r data/processed "/content/drive/MyDrive/emotion_detection_project/"
!cp -r results "/content/drive/MyDrive/emotion_detection_project/"

print("\n✅ Données sauvegardées dans Google Drive!")